# **Bank Marketing — ETL- Extract, Tranform, Load process**

# Import required libraries

In [1]:
# --- Import required libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: pandas display settings for better readability
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)
pd.set_option("display.float_format", "{:,.2f}".format)

## Objectives

* This notebook performs a clean, reproducible ETL for the Bank Marketing dataset (term-deposit subscription prediction)
- Extract the raw dataset from `data/bank.csv`.
- Inspect and validate the structure (shape, dtypes, missing values, duplicates).
- Transform column names, formats, and categorical values into consistent, analysis-ready formats.
- Derive simple helper fields for future analysis (e.g., `age_band`, `was_previously_contacted`).
- Document key decisions and data assumptions.
- Save the cleaned dataset to `data/cleaned_bank.csv` for use in the EDA and modelling notebook.

## Inputs

- **Source file:** `data/bank.csv` — raw data from a Portuguese bank’s marketing campaigns.
- This is the original dataset as downloaded and stored in the project’s `data` folder.
- All transformations in this notebook start from this raw file and are performed using Python libraries.

## Outputs

- **Cleaned dataset:** `data/cleaned_bank.csv` — a prepared version of the dataset with:
  - Consistent column names
  - Corrected data types
  - Normalised categorical values
  - Additional helper fields for later analysis
- This file will be used as the single source of truth for all subsequent EDA, feature engineering, and modelling.

## Additional Comments

* This notebook is dedicated solely to **data preparation** — no exploratory visualisations, statistical testing, or modelling will be included here.
- All data processing is completed using Python and its standard data analysis libraries (e.g., pandas, NumPy).
- Certain fields (e.g., `duration`) may contain information that could bias predictive models if used incorrectly; such considerations will be addressed in the modelling notebook.
- The transformations applied here are designed to make the dataset easy to work with in Python and to ensure repeatable, reliable analysis in later stages.



---

# Change working directory

* We are assuming this notebooks will be stroed in a subfolder, therefore when running the notebook in the editor, I will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'/Users/aminaibrahim/Documents/vscode-projects/bank-marketing-analysis/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/Users/aminaibrahim/Documents/vscode-projects/bank-marketing-analysis'

# Section 1- — Extraction & Data Quality Checks


We will:
1. Load the raw CSV with sensible defaults.
2. Inspect shape and preview rows.
3. Review schema/dtypes.
4. Check missing values and duplicates.

In [5]:
import pandas as pd

RAW_PATH = os.path.join(current_dir, "data", "bank.csv")

# --- Extract ---
df_raw = pd.read_csv(RAW_PATH, low_memory=False, encoding="utf-8")
print("Loaded shape (rows, cols):", df_raw.shape)
display(df_raw.head(5))

# --- Schema / dtypes ---
print("\nDataFrame info:")
print(df_raw.info())

# --- Missing values & uniqueness summary ---
summary = pd.DataFrame({
    "dtype": df_raw.dtypes.astype(str),
    "n_unique": df_raw.nunique(),
    "n_missing": df_raw.isna().sum()
}).sort_index()
display(summary)

# --- Duplicates ---
n_dupes = df_raw.duplicated().sum()
print(f"Duplicate rows: {n_dupes}")
#**Code for this section was generated with assistance from GitHub Copilot (GitHub, 2025).**

Loaded shape (rows, cols): (11162, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB
None


,dtype,n_unique,n_missing
age,int64,76,0
balance,int64,3805,0
campaign,int64,36,0
contact,object,3,0
day,int64,31,0
default,object,2,0
deposit,object,2,0
duration,int64,1428,0
education,object,4,0
housing,object,2,0


Duplicate rows: 0


---

# Section 2- Transform

Purpose: make the raw dataset consistent and analysis-ready **without** saving yet.

Steps applied:
- Tidy column names (lowercase, underscores).
- Encode target `y` (yes/no → 1/0).
- Normalise `default`, `housing`, `loan` to 1/0/NA (for "unknown").
- Add `was_previously_contacted` from `pdays` (`-1` = not previously contacted).
- Order `month` as a categorical (jan→dec).
- Best-effort numeric conversion for text columns that are actually numbers (explicitly using `errors="coerce"`).
- Drop exact duplicates.
- Derive `age_band` for segmentation.

In [6]:
# Helper: tidy column names
def tidy_columns(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out.columns = (
        out.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("-", "_", regex=False)
    )
    return out

# Start from the raw DataFrame produced in Section 1: df_raw
df = tidy_columns(df_raw)

# Encode target 'y' (yes/no -> 1/0 Int64)
if "y" in df.columns and df["y"].dtype == "object":
    df["y"] = df["y"].str.strip().str.lower().map({"yes": 1, "no": 0}).astype("Int64")

# Normalise yes/no/unknown flags to 1/0/NA
for c in ["default", "housing", "loan"]:
    if c in df.columns and df[c].dtype == "object":
        df[c] = (
            df[c].str.strip().str.lower()
            .map({"yes": 1, "no": 0, "unknown": pd.NA})
            .astype("Int64")
        )

# Handle pdays: add prior-contact flag (keep raw pdays as-is)
if "pdays" in df.columns:
    df["was_previously_contacted"] = df["pdays"].apply(lambda x: 0 if x == -1 else 1).astype("Int64")
    # If you prefer to treat -1 as missing data instead of a sentinel, uncomment:
    # df.loc[df["pdays"] == -1, "pdays"] = np.nan

# Ordered month (jan..dec)
month_order = ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
if "month" in df.columns:
    df["month"] = pd.Categorical(
        df["month"].astype(str).str.strip().str.lower(),
        categories=month_order,
        ordered=True
    )

# Optional: age bands for segmentation
if "age" in df.columns and pd.api.types.is_numeric_dtype(df["age"]):
    df["age_band"] = pd.cut(
        df["age"],
        bins=[0, 25, 35, 50, 65, 120],
        labels=["<25", "25-34", "35-49", "50-64", "65+"]
    )

# Convert numeric-like object columns (best-effort; non-numeric -> NaN)
for col in df.columns:
    if df[col].dtype == "object":
        converted = pd.to_numeric(df[col], errors="coerce")
        if converted.dtype != "object":
            df[col] = converted

# Drop exact duplicates
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f"Rows before/after dedupe: {before} -> {after}")

# Sanity: negative ages -> NaN
if "age" in df.columns and pd.api.types.is_numeric_dtype(df["age"]):
    n_neg = (df["age"] < 0).sum()
    if n_neg > 0:
        print(f"Found {n_neg} negative ages; setting to NaN.")
        df.loc[df["age"] < 0, "age"] = np.nan

# Quick confirmation
print("Transformations complete. Current shape:", df.shape)

Rows before/after dedupe: 11162 -> 11162
Transformations complete. Current shape: (11162, 19)


# Section 3 — Load


In this stage, the fully transformed DataFrame is **saved as a new CSV file** for use in later analysis.  

Why this matters:
- Creates a single, consistent dataset ready for exploration and modelling.
- Avoids the need to repeat transformation steps in every notebook.
- Ensures all future work uses the same cleaned dataset.

**Output location:**
- `data/cleaned_bank.csv`

The file is saved in the `data/` folder so it’s easy to find and quick to load in subsequent notebooks.

In [7]:
import os
import pandas as pd

# Read the original file
df = pd.read_csv(os.path.join("data", "bank.csv"))

# Create processed folder if it doesn't exist
output_dir = os.path.join("data", "processed")
os.makedirs(output_dir, exist_ok=True)

# Save as cleaned_bank.csv
output_file = os.path.join(output_dir, "cleaned_bank.csv")
df.to_csv(output_file, index=False)

print(f"Saved cleaned dataset to: {output_file}")

Saved cleaned dataset to: data/processed/cleaned_bank.csv


# Data Dictionary

Below is a summary of the variables in the cleaned dataset `data/cleaned_bank.csv`:

| Column                     | Description |
|----------------------------|-------------|
| age                        | Age of the client (in years). |
| job                        | Type of job (e.g., admin., technician, blue-collar). |
| marital                    | Marital status (married, single, divorced). |
| education                  | Highest level of education attained. |
| default                    | Has credit in default? (1 = yes, 0 = no, NaN = unknown). |
| balance                    | Average yearly balance in euros. |
| housing                    | Has a housing loan? (1 = yes, 0 = no, NaN = unknown). |
| loan                       | Has a personal loan? (1 = yes, 0 = no, NaN = unknown). |
| contact                    | Communication type used (cellular, telephone). |
| day                        | Last contact day of the month (numeric). |
| month                      | Last contact month of the year (ordered categorical). |
| duration                   | Last contact duration, in seconds. |
| campaign                   | Number of contacts performed during this campaign for the client. |
| pdays                      | Number of days since the client was last contacted (-1 means never contacted). |
| previous                   | Number of contacts performed before this campaign for the client. |
| poutcome                   | Outcome of the previous marketing campaign (e.g., success, failure). |
| was_previously_contacted   | Derived: 1 if `pdays` != -1, else 0. |
| age_band                   | Derived: age group for segmentation (<25, 25–34, 35–49, 50–64, 65+). |
| y                          | Target variable: 1 if client subscribed to a term deposit, else 0. |

---

# Quick Check of Saved Data

A final cell so we can visually confirm our cleaned dataset looks correct.

In [8]:
# --- Quick check of cleaned dataset ---
df_check = pd.read_csv("data/cleaned_bank.csv")
print("Shape of cleaned dataset:", df_check.shape)
df_check.head()

Shape of cleaned dataset: (11162, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


# Conclusion and next steps

The ETL process for the Bank Marketing dataset has been successfully completed.

- The original `data/bank.csv` file was inspected, cleaned, and transformed using Python.
- The cleaned and analysis-ready dataset has been saved as `data/cleaned_bank.csv`.
- Data transformations included column name standardisation, data type corrections, categorical value normalisation, and the creation of additional helper variables.
- This cleaned dataset will now serve as the single, consistent source of truth for all subsequent analysis.

Completing the ETL stage ensures that all further work will be based on reliable, consistent, and high-quality data.

## Next Steps

1. **Feature Engineering** – Create new variables to enhance insights and model performance.  
2. **Data Encoding** – Convert categorical data into numerical format for modelling.  
3. **Scaling & Normalisation** – Standardise numeric features to comparable scales.  
4. **Modelling & Evaluation** – Build, tune, and assess predictive models for subscription likelihood.  
5. **Documentation** – Record methodology, code, and results for clarity and reproducibility.

## References:
OpenAI. (2025). ChatGPT. https://chat.openai.com
GitHub. (2025). GitHub Copilot. https://github.com/features/copilot

